# 画像データの前処理

In [1]:
# データ処理ライブラリ
import numpy as np
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

# 画像処理ライブラリ
import cv2

# 可視化ライブラリ
import matplotlib.pyplot as plt

# Keras
import keras as ks
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils

# Pythonモジュール
import os
import glob

# 表示設定
pd.options.display.max_rows = 78
pd.options.display.max_columns = 78

# pathの設定
path = "./hiragana73/"

# ファイルの読み込み
charfeatures = np.load(path + 'charfeatures.npy')
files_df = pd.read_csv(path + 'files_df.csv')

Using TensorFlow backend.


In [2]:
# 画像データのサイズを確認
charfeatures.shape

(80000, 2304)

In [3]:
# files_dfの最初の5行を確認
files_df.head()

,Image,Label
0,./hiragana73/U306F\1929_1452973_0089.png,は
1,./hiragana73/U306D\1914_908938_0044.png,ね
2,./hiragana73/U308F\1926_938757_0025.png,わ
3,./hiragana73/U308B\1918_1210785_0064.png,る
4,./hiragana73/U306F\1914_1239432_0055.png,は


In [4]:
# ターゲットデータの作成
y = files_df['Label']
y.value_counts()

ま    1285
ん    1285
そ    1285
や    1285
し    1285
ふ    1285
た    1285
お    1283
わ    1283
す    1282
ゆ    1282
く    1266
さ    1261
に    1260
か    1259
を    1254
は    1247
り    1244
れ    1238
め    1233
な    1233
て    1213
あ    1208
が    1200
る    1190
も    1187
と    1184
で    1178
よ    1166
せ    1165
の    1160
け    1155
ぶ    1149
う    1148
み    1142
つ    1142
ど    1134
ぢ    1132
ね    1126
い    1122
き    1121
だ    1116
ほ    1115
こ    1115
ぜ    1115
ら    1114
へ    1114
べ    1109
ば    1105
じ    1099
ぬ    1093
ぎ    1091
づ    1080
ご    1078
え    1077
ひ    1074
ざ    1070
ろ    1069
ぞ    1066
む    1058
げ    1058
ゐ    1053
ち    1052
ず    1046
び    1045
ぼ    1044
ぐ    1043
ゑ    1030
ぺ     268
ぱ     262
ぽ     261
ぴ     126
ぷ     112
Name: Label, dtype: int64

### ターゲットクラスのひらがなの文字列を数値へ変換する

In [5]:
# ターゲットデータの前処理
encoder = LabelEncoder()
encoder.fit(y)

LabelEncoder()

In [6]:
encoder.classes_

array(['あ', 'い', 'う', 'え', 'お', 'か', 'が', 'き', 'ぎ', 'く', 'ぐ', 'け', 'げ',
       'こ', 'ご', 'さ', 'ざ', 'し', 'じ', 'す', 'ず', 'せ', 'ぜ', 'そ', 'ぞ', 'た',
       'だ', 'ち', 'ぢ', 'つ', 'づ', 'て', 'で', 'と', 'ど', 'な', 'に', 'ぬ', 'ね',
       'の', 'は', 'ば', 'ぱ', 'ひ', 'び', 'ぴ', 'ふ', 'ぶ', 'ぷ', 'へ', 'べ', 'ぺ',
       'ほ', 'ぼ', 'ぽ', 'ま', 'み', 'む', 'め', 'も', 'や', 'ゆ', 'よ', 'ら', 'り',
       'る', 'れ', 'ろ', 'わ', 'ゐ', 'ゑ', 'を', 'ん'], dtype=object)

In [7]:
# 平仮名と数字のマッピングデータを作成
encoder_character_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
print(encoder_character_mapping)

{'あ': 0, 'い': 1, 'う': 2, 'え': 3, 'お': 4, 'か': 5, 'が': 6, 'き': 7, 'ぎ': 8, 'く': 9, 'ぐ': 10, 'け': 11, 'げ': 12, 'こ': 13, 'ご': 14, 'さ': 15, 'ざ': 16, 'し': 17, 'じ': 18, 'す': 19, 'ず': 20, 'せ': 21, 'ぜ': 22, 'そ': 23, 'ぞ': 24, 'た': 25, 'だ': 26, 'ち': 27, 'ぢ': 28, 'つ': 29, 'づ': 30, 'て': 31, 'で': 32, 'と': 33, 'ど': 34, 'な': 35, 'に': 36, 'ぬ': 37, 'ね': 38, 'の': 39, 'は': 40, 'ば': 41, 'ぱ': 42, 'ひ': 43, 'び': 44, 'ぴ': 45, 'ふ': 46, 'ぶ': 47, 'ぷ': 48, 'へ': 49, 'べ': 50, 'ぺ': 51, 'ほ': 52, 'ぼ': 53, 'ぽ': 54, 'ま': 55, 'み': 56, 'む': 57, 'め': 58, 'も': 59, 'や': 60, 'ゆ': 61, 'よ': 62, 'ら': 63, 'り': 64, 'る': 65, 'れ': 66, 'ろ': 67, 'わ': 68, 'ゐ': 69, 'ゑ': 70, 'を': 71, 'ん': 72}


In [8]:
# 変更前
print('-------変更前-------')
print(y[0:5])

# ラベルエンコーダーで変換
y_encoded = encoder.transform(y)

# 変更後
print('-------変更後-------')
print(y_encoded[0:5])

-------変更前-------
0    は
1    ね
2    わ
3    る
4    は
Name: Label, dtype: object
-------変更後-------
[40 38 68 65 40]


In [9]:
# エンコードマッピングで確認
encoder_character_mapping["は"]

40

### 訓練データとテストデータに分割する

In [10]:
# 訓練/テストデータ 分割
X_train, X_test, y_train, y_test = train_test_split(charfeatures, y_encoded, train_size=10000, test_size=10000, random_state=42)
y_eval = y_test.copy()

# サイズを確認
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(10000, 2304)
(10000,)
(10000, 2304)
(10000,)


### 特徴量の正規化（Normalization）

In [11]:
# データをfloat32型へ変換
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# 正規化
X_train /= 255
X_test /= 255

### データサイズの変更
Keras のデフォルトの channels_last (batch, height, width, channles) の形式で訓練を行う。  
サイズの変更には numpy の reshape() を使う。

In [13]:
# サイズ変更
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [14]:
# ターゲットデータの確認
print(y_train[0])

61


In [15]:

# バイナリーのクラス行列へ変換
y_train = np_utils.to_categorical(y_train, 73)
y_test = np_utils.to_categorical(y_test, 73)

In [16]:
# 確認
y_train[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.], dtype=float32)